##Task 1. Modify the environments and environment groups

In [ ]:
# Add staging environment: Admin -> Environments -> Overview: + Environment ... -> Create

In [ ]:
# Add staging environment group: Admin -> Environments -> Groups: + Environment Group ...

##Task 2. Use the provisioning wizard to set up access routing

In [ ]:
# Access routing -> Edit: Enable internet access, Use wildcard DNS service, Subnetwork [api-subnet] -> Set Access

##Task 3. Create and activate a NAT address for the instance

In [ ]:
# Create NAT address: Apigee API -> organizations.instances.natAddresses
# -> create, Try this API [organizations/qwiklabs-gcp-01-8b3cab7b3a39/instances/eval-instance],
# Add request body parameters, name [apigee-nat-ip] -> Execute

In [ ]:
# Activate NAT address: Apigee API -> organizations.instances.natAddresses
# -> activate, Try this API [organizations/qwiklabs-gcp-01-8b3cab7b3a39/instances/eval-instance/natAddresses/apigee-nat-ip] -> Execute

##Task 4. Create a Cloud Armor security policy and attach it to the global load balancer

In [ ]:
# Create the Cloud Armor policy: Network security -> Cloud Armor -> Create policy: [protect-apigee], Default rule action: Allow,
# Next step -> + Add rule -> Advanced mode: Match [evaluatePreconfiguredExpr('rce-stable')], Action: Deny, Deny status: 403 (Forbidden), Priority: 1000

In [ ]:
# Attach the Cloud Armor policy: [protect-apigee] -> [three dots]: Apply policy to target -> + Add target: [apigee-proxy-backend]

##Task 5. Attach the staging environment to the runtime instance

In [ ]:
# Cloud Shell
echo ${GOOGLE_CLOUD_PROJECT}

In [ ]:
export INSTANCE_NAME=eval-instance; curl -s -H "Authorization: Bearer $(gcloud auth print-access-token)" -H "Content-Type: application/json" -X POST "https://apigee.googleapis.com/v1/organizations/${GOOGLE_CLOUD_PROJECT}/instances/${INSTANCE_NAME}/attachments" -d '{ "environment": "staging" }' | jq


In [ ]:
export ATTACHING_ENV=staging; export INSTANCE_NAME=eval-instance; echo "waiting for ${ATTACHING_ENV} attachment"; while : ; do export ATTACHMENT_DONE=$(curl -s -H "Authorization: Bearer $(gcloud auth print-access-token)" -X GET "https://apigee.googleapis.com/v1/organizations/${GOOGLE_CLOUD_PROJECT}/instances/${INSTANCE_NAME}/attachments" | jq "select(.attachments != null) | .attachments[] | select(.environment == \"${ATTACHING_ENV}\") | .environment" --join-output); [[ "${ATTACHMENT_DONE}" != "${ATTACHING_ENV}" ]] || break; echo -n "."; sleep 5; done; echo; echo "***${ATTACHING_ENV} ENVIRONMENT ATTACHED***";
